In [21]:
from vicsek import RandomSimulationConfig
import numpy as np
viscec = RandomSimulationConfig.exec_ref(RandomSimulationConfig)

viscec.step()

state0 = viscec.walkers

N = 5 # ensemble size
r = 1

In [22]:
forecast_ensemble = [viscec._step(state0) for _ in range(N)]
measurement = viscec.step()

measurement_ensemble = np.array([measurement for _ in range(N)])
noise = np.random.randn(RandomSimulationConfig.n_particles, 3)*r
virtual_observations = measurement_ensemble+noise

In [23]:
np.array(forecast_ensemble).shape

(5, 300, 3)

In [24]:
mean_forecast = np.mean(forecast_ensemble, axis = 0)
mean_forecast.shape
errors = [f-mean_forecast for f in forecast_ensemble]

In [25]:
pf = 1/(N-1) * np.sum(
    [e @ e.T for e in errors],
    axis = 0
)
pf.shape

(300, 300)

In [26]:
R = np.diag(np.ones(RandomSimulationConfig.n_particles))*r

In [29]:
K = pf*np.linalg.inv(pf+R)
K.shape

(300, 300)

In [30]:
ensemble_update = [
    x + K @ (z-x) for x, z in zip(forecast_ensemble, virtual_observations)
]

In [33]:
new_state = np.mean(ensemble_update, axis = 0)
new_state.shape

(300, 3)